## Fact Table Creation

In [0]:
from pyspark.sql.functions import *
from delta.tables import DeltaTable

In [0]:
df_silver = spark.sql('''
                      select * from parquet.`abfss://silver@carmpdatalake.dfs.core.windows.net/transformed_data`
                      ''')

In [0]:
df_dealer = spark.sql("select * from cars_catalog.gold.dim_dealer")
df_model = spark.sql("select * from cars_catalog.gold.dim_model")
df_branch = spark.sql("select * from cars_catalog.gold.dim_branch")
df_date = spark.sql("select * from cars_catalog.gold.dim_date")

In [0]:
df_fact = df_silver.join(df_dealer, df_silver['Dealer_ID'] == df_dealer['Dealer_ID'], 'left')\
                              .join(df_model, df_silver['Model_ID'] == df_model['Model_ID'], 'left')\
                              .join(df_branch, df_silver['Branch_ID'] == df_branch['Branch_ID'], 'left')\
                              .join(df_date, df_silver['Date_ID'] == df_date['Date_ID'], 'left')\
                              .select(df_silver['Revenue'], df_silver['Revenue_Per_Unit'], df_silver['Units_Sold'], df_dealer['dim_dealer_key'], df_model['dim_model_key'], df_branch['dim_branch_key'], df_date['dim_date_key'])    

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.fact_sales'):
    delta_tbl = DeltaTable.forPath(spark, 'abfss://gold@carmpdatalake.dfs.core.windows.net/fact_sales')
    delta_tbl.alias('t').merge(df_fact.alias('s'), 't.dim_dealer_key = s.dim_dealer_key AND t.dim_model_key = s.dim_model_key AND t.dim_branch_key = s.dim_branch_key AND t.dim_date_key = s.dim_date_key')\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_fact.write.format('delta')\
           .mode('overwrite')\
           .option('path', 'abfss://gold@carmpdatalake.dfs.core.windows.net/fact_sales')\
           .saveAsTable('cars_catalog.gold.fact_sales') 

In [0]:
%sql
select count(*) from cars_catalog.gold.fact_sales;